In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_path = "/content/drive/MyDrive/deepfake/KAGGLE/DATASET-balanced.csv"

audio_data = pd.read_csv(data_path)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
audio_data['LABEL'] = label_encoder.fit_transform(audio_data['LABEL'])

In [ ]:
audio_data.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.338055,0.027948,2842.948867,4322.916759,6570.586186,0.041050,-462.169586,90.311272,19.073769,24.046888,...,-6.686564,0.902086,-7.251551,-1.198342,4.747403,-4.986279,0.953935,-5.013138,-6.779060,0
1,0.443766,0.037838,2336.129597,3445.777044,3764.949874,0.047730,-409.413422,120.348808,-7.161531,5.114784,...,-2.131157,-6.876417,-1.359395,0.326401,-5.420016,-2.109968,-1.757634,-9.537907,-8.494421,0
2,0.302528,0.056578,2692.988386,2861.133180,4716.610271,0.080342,-318.996033,120.490273,-24.625771,23.891073,...,-5.853725,-3.724773,-6.627182,-5.117002,-6.072106,-0.994653,-1.617120,-3.922354,-7.033001,0
3,0.319933,0.031504,2241.665382,3503.766175,3798.641521,0.047180,-404.636749,136.320908,2.308172,-3.907071,...,-1.898315,-2.046493,-7.176277,-3.293508,4.209121,0.121835,-5.407063,-3.654926,-3.274857,0
4,0.420055,0.016158,2526.069123,3102.659519,5025.077899,0.051905,-410.497925,152.731400,-18.266771,51.993462,...,-1.952340,0.810868,6.238493,6.555839,7.535542,2.849219,2.616843,-1.793357,-5.060998,0


In [ ]:
y = audio_data['LABEL']
X = audio_data.drop('LABEL', axis = 1)

In [ ]:
X = X.values
y = y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True ,random_state=42)


In [ ]:
X_reshaped = np.zeros((X_train.shape[0], 17, 10))

for i in range(17):
    X_reshaped[:, i, :] = X_train[:, i:i + 10]

y_reshaped = np.reshape(y_train, (y_train.shape[0], 1))

X_train = X_reshaped
y_train = y_reshaped



In [ ]:
X_test_reshaped = np.zeros((X_test.shape[0], 17, 10))

for i in range(17):
    X_test_reshaped[:, i, :] = X_test[:, i:i + 10]

y_test_reshaped = np.reshape(y_test, (y_test.shape[0], 1))

X_test = X_test_reshaped
y_test = y_test_reshaped

In [ ]:
print(X_train.shape)
print(y_train.shape)


(9422, 17, 10)
(9422, 1)


**LSTM Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
model = Sequential()

model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation = 'sigmoid'))



optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss = BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 17, 64)            19200     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 56449 (220.50 KB)
Trainable params: 56449 (220.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
295/295 [==============================] - 6s 22ms/step - loss: 0.0089 - accuracy: 0.9971
Epoch 2/20
295/295 [==============================] - 8s 26ms/step - loss: 0.0070 - accuracy: 0.9976
Epoch 3/20
295/295 [==============================] - 6s 20ms/step - loss: 0.0033 - accuracy: 0.9990
Epoch 4/20
295/295 [==============================] - 8s 27ms/step - loss: 0.0093 - accuracy: 0.9964
Epoch 5/20
295/295 [==============================] - 6s 20ms/step - loss: 0.0064 - accuracy: 0.9978
Epoch 6/20
295/295 [==============================] - 7s 24ms/step - loss: 0.0028 - accuracy: 0.9993
Epoch 7/20
295/295 [==============================] - 7s 22ms/step - loss: 0.0018 - accuracy: 0.9996
Epoch 8/20
295/295 [==============================] - 6s 19ms/step - loss: 0.0027 - accuracy: 0.9993
Epoch 9/20
295/295 [==============================] - 8s 26ms/step - loss: 0.0045 - accuracy: 0.9986
Epoch 10/20
295/295 [==============================] - 6s 19ms/step - loss: 0.0095 - accura

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

74/74 [==============================] - 1s 7ms/step - loss: 0.0670 - accuracy: 0.9843
Test Loss: 0.06704764068126678
Test Accuracy: 0.9842954277992249
